In [1]:
# !pip install webrtcvad
# !pip install pygame
# !pip install pyaudio webrtcvad 
# !pip install google-cloud-texttospeech


In [2]:
from tools.initialize_groq import init_groq
from tools.file_mgmt_tools import FileOrganizerTool, MoveFileTool, CreateFolderTool, FolderMovementTool, ImprovedSearchTool
from tools.document_tools import GoogleDocWriteTool
from tools.miscellaneous_mgmt import GmailSendPdfTool, GoogleDriveUploadTool, GoogleSheetsUpdateTool

client,llm = init_groq()

In [3]:
import tools.initialize_groq
import random
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType, AgentExecutor
from google.cloud import texttospeech
from langchain.agents import load_agent


# Define the template
template = """
YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.
WHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
BE REALLY CAREFUL WITH FILE AND FOLDER ID'S! ANY WRONG ID'S WILL RESULT IN FAILURE OF OPERATIONS

If user tells you to do something that is not one of these tools/operations,\
      you kindly say that you don't have access to that functionality.

"""



# User credentials
credentials = {
    "name": "Gautham Ramachandran",
    "email": "sriramnallani35@gmail.com",
    "recemail": "gauthamramachandran3@gmail.com",
    "phone": "5715996302"
}


agent = None

In [ ]:
from flask import Flask, request, jsonify, send_file, render_template
import whisper
import pyaudio
import wave
import webrtcvad
import collections
from google.cloud import texttospeech
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from flask_cors import CORS
import requests
import logging
from tools.imports import *
import tools.initialize_groq

from dotenv import load_dotenv
my_tools = []
load_dotenv()

from langchain.agents import load_tools
humantool = load_tools(
    ["human"],
    llm=llm,
)[0]

credentials_path = os.getenv('CREDENTIALS_PATH')
my_tools.extend(
    
    [GoogleDocWriteTool(credentials_path),
    GoogleSheetsUpdateTool(credentials_path),
    GmailSendPdfTool(credentials_path),
    MoveFileTool(credentials_path),
    CreateFolderTool(credentials_path),
    FolderMovementTool(credentials_path),
    FileOrganizerTool(credentials_path),
    ImprovedSearchTool(credentials_path),
    humantool,]
)
import tools
llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(threadName)s] %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Global variables
chat_history = []
model = whisper.load_model("base")
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 1000000
vad = webrtcvad.Vad(3)
audio = pyaudio.PyAudio()
credentials = {"name": "", "email": "", "recemail": "", "phone": ""}
from flask_socketio import SocketIO, emit
from dotenv import load_dotenv
load_dotenv()
credentials_path = os.getenv('CREDENTIALS_PATH')
#tts_service_acct_path = 'C:\\Users\\THEBATMAN\\Documents\\GitHub\\RealEstateAI\\filemanager-425819-341d30387005.json'
tts_service_acct_path = os.getenv('SERVICE_ACCOUNT_PATH')
tts_client = texttospeech.TextToSpeechClient.from_service_account_file(tts_service_acct_path)

executor = ThreadPoolExecutor(max_workers=5)

socketio = SocketIO(app, cors_allowed_origins="*")

is_recording = False

@app.route('/start_recording', methods=['POST'])
def start_recording():
    global is_recording
    is_recording = True
    record_audio()
    return jsonify({"status": "recording started"})

@app.route('/stop_recording', methods=['POST'])
def stop_recording():
    global is_recording
    is_recording = False
    return jsonify({"status": "recording stopped"})

def record_audio():
    global is_recording
    logger.debug('Starting audio recording...')
    try:
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                            rate=RATE, input=True,
                            frames_per_buffer=CHUNK)
        frames = []
        ring_buffer = collections.deque(maxlen=100)
        triggered = False
        voiced_frames = []
        silence_threshold = 10
        silence_chunks = 0

        while is_recording:
            data = stream.read(CHUNK)
            frames.append(data)

            num_subframes = int(len(data) / 320)
            for i in range(num_subframes):
                subframe = data[i*320:(i+1)*320]
                is_speech = vad.is_speech(subframe, RATE)
                ring_buffer.append((subframe, is_speech))

            num_voiced = len([f for f, speech in ring_buffer if speech])

            if not triggered:
                if num_voiced > 0.6 * ring_buffer.maxlen:
                    triggered = True
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(data)
                if num_voiced < 0.2 * ring_buffer.maxlen:
                    silence_chunks += 1
                    if silence_chunks > silence_threshold:
                        triggered = False
                        break
                else:
                    silence_chunks = 0

        stream.stop_stream()
        stream.close()

        with wave.open(os.getenv('AUDIO_PATH'), 'wb') as wf:
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(audio.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(voiced_frames))
        logger.debug('Audio recording completed and file saved.')
    except Exception as e:
        logger.error(f"An error occurred while recording audio: {e}")

def transcribe_audio():
    result = model.transcribe(os.getenv('AUDIO_PATH'))
    transcription = result['text']
    
    logger.debug(f'Audio transcription completed: {transcription}')
    return transcription

async def ai_response(transcription: str):
    global chat_history
    logger.debug(f'Generating AI response for transcription: {transcription}')
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a nice, great assistant. User will tell you things. You just respond. 
                YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
                DO NOT ASSUME USER WANTS TO WRITE TO A DOCUMENT OR DO ANYTHING ELSE UNLESS YOU ARE 100% SURE!!!!!UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
                If what the user says is one of these  you must explicitly say AT THE END OF YOUR RESPONSE in this very format depending on which tool - "I will use the [google doc write tool, gmail tool, sheets tool, move file to a folder tool, create folder tool, the move folder tool, file organization tool, OR drive upload tool]". 
                so that user can confirm if you got it correctly. 
                When writing to a file, just start ONLY ONLY ONLY THE BEGINNING OF THE ENTIRE CONTENT with '@' symbol when generating file content. There should only be ONNNEEEEE '@' symbol in response and that is at the beginning!!!!!
                WHEN WRITING TO FILE--> If user indicates that they want to add on to existing content, \
                you must say 'yes append' before the aforementioned '@' symbol. OTHERWISE YOU SAY 'no append' before the aforementioned '@' symbol
                
                IMPORTANT: IF YOU REMEMBER YOU HAVE USED A TOOL ALREADY (REFER TO CHAT HISTORY), AND YOU ARE IN THE PROCESS OF USING IT AGAIN, \
                YOU MUST ASK USER IF THEY ARE SURE TO USE THE TOOL AGAIN OR NOT.

                
                If user tells you to do something that is not one of these, you kindly say that you don't have access to that functionality.
                """
            },
            {
                "role": "user",
                "content": transcription + "Here is the chat history for context (NEVER TALK ABOUT CHAT HISTORY. IT IS ONLY FOR YOU! NEVER TALK ABOUT IT IN YOUR RESPONSES!!!!): [" + str(chat_history) + "] \n\n And here is some extra info you will need"
            }
        ],
        model="llama3-70b-8192",
    )
    response = chat_completion.choices[0].message.content
    logger.debug(f'AI response generated: {response}')
    chat_history.append("USER: " + transcription + "\nTHE AI MODEL: " + response + "\n")
    
    
    asyncio.create_task(handle_response_with_agents(response))
   
    await synthesize_speech(response)

    return response

async def handle_response_with_agents(response):
    logger.debug(f'Handling response with agents: {response}')
    await asyncio.get_event_loop().run_in_executor(executor, handle_agents, response)

def handle_agents(response):
    logger.debug(f'Processing response with agents: {response}')
    name = credentials['name']
    email = credentials['email']
    recemail = credentials["recemail"]
    phone = credentials['phone']
    print(email, recemail, name, phone)

    title_of_file = ''
    with open('most_recent_file_created.txt') as mostrec:
        title_of_file = mostrec.read()
    thepath = f"./documents/{title_of_file}.pdf"

    

    if 'I will use' in response:
        llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)
        if 'google doc' in response.lower() and 'write' in response.lower():
            idx = response.find('@')
            modified_response = response[idx + 1:].strip()
            print('MODIFIED RESPONSE = ', modified_response)
            
            if('no append') in response.lower():
                append = False
            else:
                append = True
            
            print('USER EMAIL HELLLOOOOOO!!!!!!!!!!',credentials['email'])
            agent.invoke({"input": f"use ONLY google doc write tool!!! INPUT EVERYTHING CORRECTLY EVEN IF ITS LONG!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! append-->true or false lowercase only!!! NO DICTIONARIES, JUST EVERYTHING PASSED AS IT ISSS!!!!!--> INPUT_TEXT: {modified_response} | APPEND: {append} |  USER_EMAIL: {credentials['email']} ||| PLEASE OUTPUT THE EXACT NAME OF THE GOOGLE DOC THAT WAS CREATED"})
        elif 'sheet' in response:
            agent.invoke({"input": f"please ONLY ONLY use google sheets update tool, input ONLY ONLY these into the tool PROPERLY! --> {credentials['name']}, {credentials['phone']}"})
        elif 'upload' in response:
            agent.invoke({"input": f"please use google drive upload tool, INPUTS --> FILE_PATH: {thepath} USER_EMAIL: {credentials['email']}"})
        elif 'mail' in response:
            if 'pdf' in response or 'PDF' in response or 'file' in response:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}"})
            else:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing"})
        elif 'create' in response and 'folder' in response:
            agent.invoke({"input": f"please ONLY use create folder tool!!! INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER NAME: the folder name as specified in response --> {response} ||| PLEASE OUTPUT THE EXACT NAME OF THE FOLDER THAT WAS CREATED"})
        elif 'move' in response and 'file' in response:
            agent.invoke({"input": f"please ONLY use move file tool!!! INPUT DIRECTLY THE FOLLOWING INPUTS --> FILE_NAME please deduce from the response and then pass it into tool: {response} ; FOLDER_NAME deduce from the response and then pass it into tool - PASS IN 'root' if user says move into google drive or my drive: {response}"})
        elif 'folder' in response and 'move' in response:
            agent.invoke({"input": f"please ONLY use move folder tool!!! INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER_NAME please deduce from the response and then pass it into tool: {response} ; NEW_PARENT_FOLDER_NAME also deduce from response - PASS IN 'root' if user says move into google drive or my drive: {response}"})
        elif 'file' in response and 'organization' in response:
            agent.invoke({"input": f"please ONLY use file organization tool!!! INPUT DIRECTLY THE FOLLOWING INPUTS --> PARENT_FOLDER_NAME please deduce from the response and then pass it into tool: {response}"})


async def synthesize_speech(text):
    logger.debug(f'Starting speech synthesis for text: {text}')
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Casual-K"
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = await asyncio.get_event_loop().run_in_executor(
        executor, lambda: tts_client.synthesize_speech(
            input=synthesis_input, voice=voice, audio_config=audio_config
        )
    )
    async with aiofiles.open(os.getenv('TTS_SYNTHESIS'), 'wb') as out:
        await out.write(response.audio_content)
    print('Audio content written to file "synthesis.mp3"')
    logger.debug('Speech synthesis completed and file saved.')


@app.route('/set_credentials', methods=['POST'])
def set_credentials():
    global credentials
    data = request.get_json()
    if not data:
        return jsonify({"status": "failed", "message": "No data received"}), 400
    credentials['name'] = data.get('name')
    credentials['email'] = data.get('email')
    credentials['recemail'] = data.get('recemail')
    credentials['phone'] = data.get('phone')
    logger.info("THE CREDENTIALS ****** -------------> ", credentials)
    return jsonify({"status": "success"})


@app.route('/')
def index():
    return render_template('index2.html')

@app.route('/voice_assistant')
def voice_assistant():
    return render_template('index2.html')


@app.route('/authenticate', methods=['POST'])
def authenticate():
    auth_header = request.headers.get('Authorization')
    token = auth_header.split(' ')[1] if auth_header else None

    if not token:
        return jsonify({'error': 'Missing token'}), 400

    response = requests.get(
        'https://www.googleapis.com/oauth2/v3/userinfo',
        headers={'Authorization': f'Bearer {token}'}
    )

    if response.status_code != 200:
        return jsonify({'error': 'Failed to fetch user info'}), response.status_code

    user_info = response.json()
    return jsonify(user_info), 200

@app.route('/talk', methods=['POST'])
async def talk():
    loop = asyncio.get_event_loop()
    
    # Ensure recording is stopped before proceeding
    global is_recording
    if is_recording:
        return jsonify({"error": "Recording is still in progress"}), 400
    
    logger.debug('Starting audio transcription...')
    transcription = await loop.run_in_executor(executor, transcribe_audio)
    logger.debug(f'Audio transcription completed: {transcription}')
    
    logger.debug('Generating AI response...')
    ai_resp = await ai_response(transcription)
    logger.debug(f'AI response generated: {ai_resp}')
    
    return jsonify({'response': ai_resp})


@app.route('/get_audio')
def get_audio():
    return send_file(os.getenv('TTS_SYNTHESIS'), mimetype="audio/mp3")

if __name__ == '__main__':
    app.run()